# Import Required Libraries
Import necessary libraries such as NumPy, VTK, and vmtk.

In [2]:
# Import Required Libraries
import numpy as np
import vtk
import vmtk
from vmtk import vtkvmtk
from vmtk.vmtkscripts import vmtkCenterlines
from vmtk.vmtkscripts import vmtkPointListSeedSelector
import vmtk.vmtkscripts as vmtkscripts
from vmtk import pypes

# Define Helper Functions
Define all the helper functions from the provided code, including `point_cloud_to_watertight_mesh`, `mesh_to_volume`, `visualize_mesh_with_centerline`, `slice_mesh_and_connect_centroids`, and others.

In [3]:
# Helper Functions

def point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=1.0):
    """
    Converts a point cloud (.npy) to a watertight mesh using alpha shapes.

    Parameters:
        point_cloud_file (str): Path to the input point cloud (.npy).
        output_mesh_file (str): Path to save the output watertight mesh (.vtp).
        alpha (float): Alpha parameter for vtkDelaunay3D.
    """
    # Load the point cloud
    points = np.load(point_cloud_file)
    print("Shape of point cloud:", points.shape)
    if points.shape[1] != 3:
        raise ValueError("Point cloud must have shape (N, 3).")

    # Create vtkPoints and add the points to it
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a vtkPolyData object to store the points
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)

    # Generate the Delaunay tessellation
    delaunay = vtk.vtkDelaunay3D()
    delaunay.SetInputData(poly_data)
    delaunay.SetAlpha(alpha)
    delaunay.Update()

    # Extract the surface of the tessellation
    surface_filter = vtk.vtkDataSetSurfaceFilter()
    surface_filter.SetInputConnection(delaunay.GetOutputPort())
    surface_filter.Update()

    # Write the watertight mesh to a file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_mesh_file)
    writer.SetInputConnection(surface_filter.GetOutputPort())
    writer.Write()

    print(f"Watertight mesh saved to: {output_mesh_file}")

def mesh_to_volume(input_mesh_file, volume_file, spacing=(1.0, 1.0, 1.0)):
    """Converts a watertight mesh to a volumetric representation (e.g., voxel grid)."""
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_mesh_file)
    reader.Update()
    bounds = reader.GetOutput().GetBounds()
    image_data = vtk.vtkImageData()
    image_data.SetSpacing(spacing)
    dims = [
        int((bounds[1] - bounds[0]) / spacing[0]),
        int((bounds[3] - bounds[2]) / spacing[1]),
        int((bounds[5] - bounds[4]) / spacing[2]),
    ]
    image_data.SetDimensions(dims)
    image_data.SetOrigin(bounds[0], bounds[2], bounds[4])
    stencil = vtk.vtkPolyDataToImageStencil()
    stencil.SetInputConnection(reader.GetOutputPort())
    stencil.SetOutputSpacing(spacing)
    stencil.SetOutputOrigin(bounds[0], bounds[2], bounds[4])
    stencil.SetOutputWholeExtent(image_data.GetExtent())
    stencil.Update()
    volume = vtk.vtkImageStencil()
    volume.SetInputData(image_data)
    volume.SetStencilConnection(stencil.GetOutputPort())
    volume.ReverseStencilOff()
    volume.SetBackgroundValue(0)
    volume.Update()
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(volume_file)
    writer.SetInputConnection(volume.GetOutputPort())
    writer.Write()
    print(f"Volume saved to: {volume_file}")

def clean_and_connect_mesh(input_mesh):
    """Cleans the mesh and extracts the largest connected component."""
    connectivity_filter = vtk.vtkConnectivityFilter()
    connectivity_filter.SetInputData(input_mesh)
    connectivity_filter.SetExtractionModeToLargestRegion()
    connectivity_filter.Update()
    return connectivity_filter.GetOutput()

def smooth_mesh(input_mesh, iterations=30, relaxation_factor=0.1):
    """Smooths the mesh to improve quality."""
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputData(input_mesh)
    smoother.SetNumberOfIterations(iterations)
    smoother.SetRelaxationFactor(relaxation_factor)
    smoother.FeatureEdgeSmoothingOff()
    smoother.BoundarySmoothingOn()
    smoother.Update()
    return smoother.GetOutput()

def generate_voronoi_from_mesh(mesh_file, voronoi_output_file):
    """
    Generates a Voronoi diagram based on the points of a watertight mesh.
    """
    # Read the watertight mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Debug: Check the number of points and cells in the mesh
    print("Number of points in mesh:", mesh.GetNumberOfPoints())
    print("Number of cells in mesh:", mesh.GetNumberOfCells())
    if mesh.GetNumberOfPoints() == 0 or mesh.GetNumberOfCells() == 0:
        raise ValueError("Input mesh is empty or invalid.")

    # Ensure the input is vtkPolyData
    if not isinstance(mesh, vtk.vtkPolyData):
        print("Converting input to vtkPolyData using vtkGeometryFilter...")
        geometry_filter = vtk.vtkGeometryFilter()
        geometry_filter.SetInputData(mesh)
        geometry_filter.Update()
        mesh = geometry_filter.GetOutput()

    # Clean the mesh
    cleaner = vtk.vtkCleanPolyData()
    cleaner.SetInputData(mesh)
    cleaner.Update()
    mesh = cleaner.GetOutput()

    # Debug: Check the cleaned mesh
    print("Number of points in cleaned mesh:", mesh.GetNumberOfPoints())
    print("Number of cells in cleaned mesh:", mesh.GetNumberOfCells())
    if mesh.GetNumberOfPoints() == 0 or mesh.GetNumberOfCells() == 0:
        raise ValueError("Cleaned mesh is empty or invalid.")

    # Extract points from the mesh
    points = mesh.GetPoints()

    # Create a polydata object to store the points
    point_polydata = vtk.vtkPolyData()
    point_polydata.SetPoints(points)

    # Generate the Delaunay tessellation
    delaunay_filter = vtk.vtkDelaunay3D()
    delaunay_filter.SetInputData(point_polydata)
    delaunay_filter.Update()
    delaunay_output = delaunay_filter.GetOutput()

    # Debug: Check the number of cells and points in the Delaunay tessellation
    print("Number of cells in Delaunay tessellation:", delaunay_output.GetNumberOfCells())
    print("Number of points in Delaunay output:", delaunay_output.GetNumberOfPoints())
    if delaunay_output.GetNumberOfCells() == 0:
        raise ValueError("Delaunay tessellation failed.")

    # Write the Delaunay tessellation to a file (as a substitute for Voronoi)
    writer = vtk.vtkXMLUnstructuredGridWriter()
    writer.SetFileName(voronoi_output_file)
    writer.SetInputData(delaunay_output)
    writer.Write()

    # Debug: Check if the Voronoi file was written
    import os
    if not os.path.exists(voronoi_output_file):
        raise FileNotFoundError(f"Voronoi file not found: {voronoi_output_file}")
    print(f"Voronoi file size: {os.path.getsize(voronoi_output_file)} bytes")

    print(f"Voronoi diagram saved to: {voronoi_output_file}")

def visualize_voronoi(voronoi_file):
    reader = vtk.vtkXMLUnstructuredGridReader()
    reader.SetFileName(voronoi_file)
    reader.Update()
    voronoi = reader.GetOutput()

    mapper = vtk.vtkDataSetMapper()
    mapper.SetInputData(voronoi)

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    render_window.Render()
    render_window_interactor.Start()

def extract_centerline_from_mesh(mesh_file, centerline_output_file):
    """
    Extracts the centerline directly from a surface mesh using VMTK.

    Parameters:
        mesh_file (str): Path to the surface mesh file (.vtp).
        centerline_output_file (str): Path to save the extracted centerline (.vtp).
    """
    import os
    from vmtk.vmtkscripts import vmtkCenterlines
    import vtk

    if not os.path.exists(mesh_file):
        raise FileNotFoundError(f"Mesh file not found: {mesh_file}")
    print(f"Reading mesh file: {mesh_file}")

    # Read the surface mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    surface_mesh = reader.GetOutput()

    # Debug: Check mesh validity
    print("Number of points in mesh:", surface_mesh.GetNumberOfPoints())
    if surface_mesh.GetNumberOfPoints() == 0:
        raise ValueError("Surface mesh is empty or invalid.")

    # Automatically determine source and target points based on bounds
    bounds = surface_mesh.GetBounds()
    print("Bounds of mesh:", bounds)

    source_point = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, bounds[4]]  # Bottom center
    target_point = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, bounds[5]]  # Top center

    # Snap source and target points to the surface
    point_locator = vtk.vtkPointLocator()
    point_locator.SetDataSet(surface_mesh)
    point_locator.BuildLocator()

    source_id = point_locator.FindClosestPoint(source_point)
    target_id = point_locator.FindClosestPoint(target_point)

    snapped_source_point = surface_mesh.GetPoint(source_id)
    snapped_target_point = surface_mesh.GetPoint(target_id)

    # Explicitly create SourcePoints and TargetPoints as flat lists
    source_points = [float(snapped_source_point[0]), float(snapped_source_point[1]), float(snapped_source_point[2])]
    target_points = [float(snapped_target_point[0]), float(snapped_target_point[1]), float(snapped_target_point[2])]

    print("Source point:", source_points)
    print("Target point:", target_points)

    # Use VMTK to compute centerlines
    centerlines = vmtkCenterlines()
    centerlines.Surface = surface_mesh
    centerlines.SeedSelectorName = 'pointlist'
    centerlines.SourcePoints = source_points
    centerlines.TargetPoints = target_points
    print("Starting centerline extraction...")
    try:
        centerlines.Execute()
        print("Centerline extraction completed.")

        # Write the centerline to a file
        writer = vtk.vtkXMLPolyDataWriter()
        writer.SetFileName(centerline_output_file)
        writer.SetInputData(centerlines.Centerlines)
        writer.Write()
        print(f"Centerline saved to: {centerline_output_file}")

    except Exception as e:
        print(f"Error during centerline extraction: {e}")

def visualize_centerline(centerline_file, mesh_file=None):
    """
    Visualizes the centerline and optionally the mesh together.
    
    Parameters:
        centerline_file (str): Path to the centerline file (.vtp)
        mesh_file (str, optional): Path to the mesh file (.vtp)
    """
    # Read the centerline file
    centerline_reader = vtk.vtkXMLPolyDataReader()  # Correct reader
    centerline_reader.SetFileName(centerline_file)
    centerline_reader.Update()
    centerline = centerline_reader.GetOutput()
    
    # Create a mapper and actor for the centerline
    centerline_mapper = vtk.vtkPolyDataMapper()
    centerline_mapper.SetInputData(centerline)
    centerline_actor = vtk.vtkActor()
    centerline_actor.SetMapper(centerline_mapper)
    centerline_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red
    centerline_actor.GetProperty().SetLineWidth(3)
    
    # Create renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)
    
    # Add centerline actor to renderer
    renderer.AddActor(centerline_actor)
    
    # If a mesh file is provided, add it to the visualization
    if mesh_file:
        mesh_reader = vtk.vtkXMLPolyDataReader()
        mesh_reader.SetFileName(mesh_file)
        mesh_reader.Update()
        mesh = mesh_reader.GetOutput()
        
        mesh_mapper = vtk.vtkPolyDataMapper()
        mesh_mapper.SetInputData(mesh)
        mesh_actor = vtk.vtkActor()
        mesh_actor.SetMapper(mesh_mapper)
        mesh_actor.GetProperty().SetColor(0.8, 0.8, 0.8)  # Light gray
        mesh_actor.GetProperty().SetOpacity(0.5)  # Semi-transparent
        
        renderer.AddActor(mesh_actor)
    
    # Set background color and start visualization
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background
    render_window.Render()
    render_window_interactor.Start()

def compute_gradient(arrival_times, point):
    """
    Computes the gradient of the arrival times at a given point.
    """
    # Placeholder for gradient computation
    pass

def move_along_gradient(point, gradient):
    """
    Moves a point along the gradient.
    """
    # Placeholder for movement along the gradient
    pass

def visualize_mesh_with_centerline(mesh_file, centerline_file):
    # Read the mesh file
    mesh_reader = vtk.vtkXMLPolyDataReader()
    mesh_reader.SetFileName(mesh_file)
    mesh_reader.Update()
    mesh = mesh_reader.GetOutput()

    # Read the centerline file
    centerline_reader = vtk.vtkXMLPolyDataReader()
    centerline_reader.SetFileName(centerline_file)
    centerline_reader.Update()
    centerline = centerline_reader.GetOutput()

    # Create a mapper and actor for the mesh
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)
    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetColor(0.8, 0.8, 0.8)  # Light gray

    # Create a mapper and actor for the centerline
    centerline_mapper = vtk.vtkPolyDataMapper()
    centerline_mapper.SetInputData(centerline)
    centerline_actor = vtk.vtkActor()
    centerline_actor.SetMapper(centerline_mapper)
    centerline_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red
    centerline_actor.GetProperty().SetLineWidth(3)

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actors to the renderer
    renderer.AddActor(mesh_actor)
    renderer.AddActor(centerline_actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def slice_mesh_and_connect_centroids(mesh_file, num_slices=10):
    """
    Slices the mesh into a specified number of slices along the Z-axis and connects the centroids of each slice.
    """
    # Read the mesh file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Get the bounds of the mesh
    bounds = mesh.GetBounds()
    z_min, z_max = bounds[4], bounds[5]
    slice_height = (z_max - z_min) / num_slices

    # Initialize a list to store centroids
    centroids = []

    # Slice the mesh and compute centroids
    for i in range(num_slices):
        z_lower = z_min + i * slice_height
        z_upper = z_lower + slice_height

        # Create a plane to slice the mesh
        plane = vtk.vtkPlane()
        plane.SetOrigin(0, 0, z_lower)
        plane.SetNormal(0, 0, 1)

        # Clip the mesh with the plane
        clipper = vtk.vtkClipPolyData()
        clipper.SetInputData(mesh)
        clipper.SetClipFunction(plane)
        clipper.Update()
        sliced_mesh = clipper.GetOutput()

        # Compute the centroid of the sliced mesh
        center_of_mass_filter = vtk.vtkCenterOfMass()
        center_of_mass_filter.SetInputData(sliced_mesh)
        center_of_mass_filter.Update()
        centroid = center_of_mass_filter.GetCenter()
        centroids.append(centroid)

    # Create a polyline to connect the centroids
    points = vtk.vtkPoints()
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(len(centroids))

    for i, centroid in enumerate(centroids):
        points.InsertNextPoint(centroid)
        polyline.GetPointIds().SetId(i, i)

    # Create a polydata to store the polyline
    polyline_data = vtk.vtkPolyData()
    polyline_data.SetPoints(points)
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)
    polyline_data.SetLines(cells)

    # Visualize the polyline
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(polyline_data)
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Add the actor to the renderer
    renderer.AddActor(actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def point_cloud_to_vtp(point_cloud_file, output_vtp_file):
    """
    Converts a point cloud (.npy) to a .vtp file for visualization in VTK.
    """
    # Load the point cloud from the .npy file
    points = np.load(point_cloud_file)

    # Create a vtkPoints object and add the points to it
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a vtkPolyData object to store the points
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)

    # Write the vtkPolyData to a .vtp file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_vtp_file)
    writer.SetInputData(poly_data)
    writer.Write()

    print(f"Point cloud saved to: {output_vtp_file}")

# Convert Point Cloud to Watertight Mesh
Run the `point_cloud_to_watertight_mesh` function with the specified input and output file paths.

In [4]:
# Input and output file paths
import numpy as np
import vtk
point_cloud_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\accumulated_point_cloud_frame_100.npy"
output_mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"

# Convert point cloud to watertight mesh
point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=1.0)

Shape of point cloud: (7844, 3)
Watertight mesh saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp


# Generate Voronoi
Run the `generate_voronoi_from_mesh` function to extract the voronoi from the watertight mesh. 

In [5]:
# File paths
mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"
voronoi_output_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\voronoi_output.vtu"

# Generate Voronoi diagram from watertight mesh
generate_voronoi_from_mesh(mesh_file, voronoi_output_file)

# Visualize the Voronoi diagram
visualize_voronoi(voronoi_output_file)

Number of points in mesh: 2936
Number of cells in mesh: 9509
Number of points in cleaned mesh: 2936
Number of cells in cleaned mesh: 9509
Number of cells in Delaunay tessellation: 18190
Number of points in Delaunay output: 2936
Voronoi file size: 255547 bytes
Voronoi diagram saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\voronoi_output.vtu


# Extract Centerline
Run the `extract_centerline_from_voronoi` function to extract the centerline from the voronoi.

In [6]:
import os
from vmtk import vmtkscripts
import vtk

# File paths
mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"
centerline_output_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\centerline_from_mesh.vtp"

# Check if the mesh file exists
if not os.path.exists(mesh_file):
    raise FileNotFoundError(f"Mesh file not found: {mesh_file}")

# Correct reader for .vtp files
reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName(mesh_file)
reader.Update()
surface = reader.GetOutput()

if surface is None or surface.GetNumberOfCells() == 0:
    raise ValueError(f"Could not read or surface is empty: {mesh_file}")

# Initialize the vmtkCenterlines script
centerlines = vmtkscripts.vmtkCenterlines()
centerlines.Surface = surface
centerlines.AdvancementRatio = 1.05  # Use the default or your desired value
# If your mesh has clear inlets/outlets, you might not need to set SourcePoints/TargetPoints
# centerlines.SeedSelectorName = 'openprofiles'
# If you want to specify start/end points (you'd need to identify them on your mesh)
# centerlines.SeedSelectorName = 'pointlist'
# centerlines.SourcePoints = [[x1, y1, z1]]
# centerlines.TargetPoints = [[x2, y2, z2]]

print("Starting centerline extraction...")
centerlines.Execute()
print("Centerline extraction completed.")

# Write the centerline to a file
writer = vtk.vtkPolyDataWriter()
writer.SetFileName(centerline_output_file)
writer.SetInputData(centerlines.Centerlines)
writer.Write()

print(f"Centerline saved to: {centerline_output_file}")

Starting centerline extraction...
Cleaning surface.
Triangulating surface.
Capping surface.
Please position the mouse and press space to add source points, 'u' to undo
Quit renderer
Please position the mouse and press space to add target points, 'u' to undo
Quit renderer
Computing centerlines.
Computing centerlines...Centerline extraction completed.
Centerline saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\centerline_from_mesh.vtp


# Visualize Mesh and Centerline
Run the `visualize_mesh_with_centerline` function to visualize the mesh and its centerline together.

In [ ]:
# Visualize the mesh and centerline together
visualize_mesh_with_centerline(output_mesh_file, centerline_output_file)

: 

# Slice Mesh and Connect Centroids
Run the `slice_mesh_and_connect_centroids` function to slice the mesh and connect centroids with a line.

In [7]:
# Slice the mesh and connect centroids
slice_mesh_and_connect_centroids(output_mesh_file, num_slices=10)

# Convert Point Cloud to VTP
Run the `point_cloud_to_vtp` function to convert the point cloud to a `.vtp` file.

In [8]:
# Convert the point cloud to .vtp
point_cloud_to_vtp(point_cloud_file, output_mesh_file)

Point cloud saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp
